<a href="https://colab.research.google.com/github/gopal2812/mlblr/blob/master/Assignment7Sentiment_Analysis_using_LSTM_RNN_data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0 TorchText

## Dataset Preview

Your first step to deep learning in NLP. We will be mostly using PyTorch. Just like torchvision, PyTorch provides an official library, torchtext, for handling text-processing pipelines. 

We will be using standford sentiment Treebank dataset.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')



Mounted at /content/gdrive/


In [2]:
!ls /content/gdrive/MyDrive/end/stanfordSentimentTreebank.zip
!cp '/content/gdrive/MyDrive/end/stanfordSentimentTreebank.zip' stanfordSentimentTreebank.zip
!unzip -q -o stanfordSentimentTreebank.zip 

/content/gdrive/MyDrive/end/stanfordSentimentTreebank.zip


In [3]:
#!rm -rf stanfordSentimentTreebank
!ls -l stanfordSentimentTreebank/

total 19920
-rwxr-xr-x 1 root root  1290263 Oct  9  2013 datasetSentences.txt
-rwxr-xr-x 1 root root    83764 Oct  9  2013 datasetSplit.txt
-rwxr-xr-x 1 root root 12010637 Oct  9  2013 dictionary.txt
-rwxr-xr-x 1 root root  1195613 Feb  2  2013 original_rt_snippets.txt
-rwxr-xr-x 1 root root     2357 Oct  9  2013 README.txt
-rwxr-xr-x 1 root root  3263577 Oct  9  2013 sentiment_labels.txt
-rwxr-xr-x 1 root root  1226029 Feb  2  2013 SOStr.txt
-rwxr-xr-x 1 root root  1308918 Feb  2  2013 STree.txt


In [4]:
import pandas as pd
df = pd.read_csv('stanfordSentimentTreebank/datasetSentences.txt',sep='\t')
df.tail()


,sentence_index,sentence
11850,11851,A real snooze .
11851,11852,No surprises .
11852,11853,We 've seen the hippie-turned-yuppie plot befo...
11853,11854,Her fans walked out muttering words like `` ho...
11854,11855,In this case zero .


In [5]:
df = pd.read_csv('stanfordSentimentTreebank/datasetSplit.txt',sep='\t')
df.tail()


,"sentence_index,splitset_label"
11850,"11851,1"
11851,"11852,1"
11852,"11853,1"
11853,"11854,1"
11854,"11855,1"


In [6]:
df = pd.read_csv('stanfordSentimentTreebank/SOStr.txt',sep='\t')
df.tail()


,"The|Rock|is|destined|to|be|the|21st|Century|'s|new|``|Conan|''|and|that|he|'s|going|to|make|a|splash|even|greater|than|Arnold|Schwarzenegger|,|Jean-Claud|Van|Damme|or|Steven|Segal|."
11849,A|real|snooze|.
11850,No|surprises|.
11851,We|'ve|seen|the|hippie-turned-yuppie|plot|befo...
11852,Her|fans|walked|out|muttering|words|like|``|ho...
11853,In|this|case|zero|.


In [7]:
df = pd.read_csv('stanfordSentimentTreebank/sentiment_labels.txt',sep='\t')
df.tail()

,phrase ids|sentiment values
239227,239227|0.36111
239228,239228|0.38889
239229,239229|0.33333
239230,239230|0.88889
239231,239231|0.5


In [8]:
df = pd.read_csv('stanfordSentimentTreebank/dictionary.txt',sep='\t')
df.tail()

,!|0
239226,zoning ordinances to protect your community fr...
239227,zzzzzzzzz|179256
239228,élan|220442
239229,É|220443
239230,É um passatempo descompromissado|220444


## Prepare the labels into 25 category

In [9]:
import sys
import pandas as pd

dir_corpus = ''

def read_text():
  ifname = dir_corpus + 'stanfordSentimentTreebank/SOStr.txt'
  lines = open(ifname, 'r').read().split('\n')

  texts = []
  for line in lines:
    params = line.split('|')
    if len(params) > 1:
      text = ' '.join(params)
      texts.append(text)

  return texts

def read_splitlabel():
  ifname = dir_corpus + 'stanfordSentimentTreebank/datasetSplit.txt'
  lines = open(ifname, 'r').read().split('\n')

  splitlabels = []
  for line in lines[1:]:
    params = line.split(',')
    if len(params) == 2:
      splitlabels.append(int(params[1]))
	
  return splitlabels

def read_sentiscore():
  ifname = dir_corpus + 'stanfordSentimentTreebank/sentiment_labels.txt'
  lines = open(ifname, 'r').read().split('\n')

  sentiscores = []
  for line in lines[1:]:
    params = line.split('|')
    if len(params) == 2:
      sentiscores.append(float(params[1]))

  return sentiscores

def read_phraseid():
  ifname = dir_corpus + 'stanfordSentimentTreebank/dictionary.txt'
  lines = open(ifname, 'r').read().split('\n')

  phraseid = {}
  for line in lines:
    params = line.split('|')
    if len(params) == 2:
      phraseid[params[0]] = int(params[1])

  return phraseid

**Data Augmentation routine **

In [10]:
!!pip install google_trans_new


['Collecting google_trans_new',
 '  Downloading https://files.pythonhosted.org/packages/f9/7b/9f136106dc5824dc98185c97991d3cd9b53e70a197154dd49f7b899128f6/google_trans_new-1.1.9-py3-none-any.whl',
 'Installing collected packages: google-trans-new',
 'Successfully installed google-trans-new-1.1.9']

In [11]:
import google_trans_new
from google_trans_new import google_translator

def backTranslation(sentence):
  translator = google_translator()
  available_langs = list(google_trans_new.LANGUAGES.keys()) 
  trans_lang = random.choice(available_langs) 
  translations = translator.translate(sentence, lang_tgt=trans_lang) 
  translations_en_random = translator.translate(translations, lang_src=trans_lang, lang_tgt='en') 
  return translations_en_random

In [21]:
def random_swap(sentence, n=5): 
    #print(sentence)
    sentence = sentence.split()
    length = range(len(sentence))
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    sentence = ' '.join(sentence)
    return sentence

def random_deletion(words, p=0.5): 
    words = words.split()
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return ' '.join(random.choice(words))
    else:
        return ' '.join(remaining)



def augment_data(x_label):
  x, label = x_label
  x_temp = x
  label_temp = label
  counter = 0
  for i in range(len(x)):
    if counter/500 == 0:
      print(counter)
    counter = counter +1
    #print(x[i])
    try:
      t = random_swap(x[i])
      x_temp.append(t)
      label_temp.append(label[i])
      td = random_deletion(x[i])
      x_temp.append(td)
      label_temp.append(label[i])
      #tb = backTranslation(x[i])
      #x_temp.append(tb)
      #label_temp.append(label[i])
    except:
      continue

  return(x_temp,label_temp)

#augment_data((["hi i am good very happy","i am"], [2,3]))
    


Data Lbel routine creation

In [22]:
def labelize(x_label):
  x, label = x_label
  y = []
  for l in label:
    if l <= 0.04:
      y.append(1)
    elif l <= 0.08:
      y.append(2)
    elif l <= 0.12:
      y.append(3)
    elif l <= 0.16:
      y.append(4)
    elif l <= 0.20:
      y.append(5)
    elif l <= 0.24:
      y.append(6)
    elif l <= 0.28:
      y.append(7)
    elif l <= 0.32:
      y.append(8)
    elif l <= 0.36:
      y.append(9)
    elif l <= 0.40:
      y.append(10)
    elif l <= 0.44:
      y.append(11)
    elif l <= 0.48:
      y.append(12)
    elif l <= 0.52:
      y.append(13)
    elif l <= 0.56:
      y.append(14)
    elif l <= 0.60:
      y.append(15)
    elif l <= 0.64:
      y.append(16)
    elif l <= 0.68:
      y.append(17)
    elif l <= 0.72:
      y.append(18)
    elif l <= 0.76:
      y.append(19)
    elif l <= 0.80:
      y.append(20)
    elif l <= 0.84:
      y.append(21)
    elif l <= 0.88:
      y.append(22)
    elif l <= 0.92:
      y.append(23)
    elif l <= 0.96:
      y.append(24)
    else:
      y.append(25)

  print(len(y))
  return (x, y)

In [23]:
def prepare_label():
  texts = read_text()
  splitlabels = read_splitlabel()
  sentiscores = read_sentiscore()
  phraseid = read_phraseid()

  train_text = []
  train_label = []
	
  valid_text = []
  valid_label = []

  test_text = []
  test_label = []

  n_sample = len(texts)
  if n_sample == len(splitlabels) and len(sentiscores) == len(phraseid):
    print('%d samples'%(n_sample))
  else:
    print('reading fail')
	#for i, didx in enumerate(splitlabels):
	#	print(didx)
  for i, didx in enumerate(splitlabels):
    if didx == 1:
      list_text = train_text
      list_label = train_label
    elif didx == 3:
      #print("3")
      list_text = valid_text
      list_label = valid_label
    elif didx == 2:
      list_text = test_text
      list_label = test_label
    list_text.append(texts[i])
    list_label.append(sentiscores[phraseid[texts[i]]])
  print(len(train_text), len(train_label))
  return (augment_data(labelize((train_text, train_label))), labelize((valid_text, valid_label)), labelize((test_text, test_label)))

In [24]:
import random
(train, train_label), (valid, valid_label), (test, test_label) = prepare_label()
set(train_label)

11855 samples
8544 8544
8544
0
1101
2210


{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25}

In [26]:
(len(train), len(valid), len(test))
len(train_label)

25632

## Defining Fields

Now we shall be defining LABEL as a LabelField, which is a subclass of Field that sets sequen tial to False (as it’s our numerical category class). SENTENCE is a standard Field object, where we have decided to use the spaCy tokenizer and convert all the text to lower‐ case.

In [27]:
# Import Library
import random
import torch, torchtext
from torchtext import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [28]:
Sentence = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

print(Sentence)

Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the CSV:

In [29]:
fields = [('sentence', Sentence),('labels',Label)]

Armed with our declared fields, lets convert from pandas to list to torchtext. We could also use TabularDataset to apply that definition to the CSV directly but showing an alternative approach too.

In [30]:
example_train = [data.Example.fromlist([train[i], train_label[i]], fields) for i in range(len(train))]
example_valid = [data.Example.fromlist([valid[i], valid_label[i]], fields) for i in range(len(valid))] 

In [31]:
# Creating dataset
#twitterDataset = data.TabularDataset(path="tweets.csv", format="CSV", fields=fields, skip_header=True)

trainDataset = data.Dataset(example_train, fields)
validDataset = data.Dataset(example_valid, fields)

Finally, we can split into training, testing, and validation sets by using the split() method:

In [32]:
(len(trainDataset), len(validDataset))

(25632, 1101)

An example from the dataset:

In [33]:
vars(trainDataset.examples[10])

{'labels': 23,
 'sentence': ['Good',
  'fun',
  ',',
  'good',
  'action',
  ',',
  'good',
  'acting',
  ',',
  'good',
  'dialogue',
  ',',
  'good',
  'pace',
  ',',
  'good',
  'cinematography',
  '.']}

## Building Vocabulary

At this point we would have built a one-hot encoding of each word that is present in the dataset—a rather tedious process. Thankfully, torchtext will do this for us, and will also allow a max_size parameter to be passed in to limit the vocabu‐ lary to the most common words. This is normally done to prevent the construction of a huge, memory-hungry model. We don’t want our GPUs too overwhelmed, after all. 

Let’s limit the vocabulary to a maximum of 5000 words in our training set:


In [34]:
#Tweet.build_vocab(train)
#Label.build_vocab(train)
MAX_VOCAB_SIZE = 25_000

Sentence.build_vocab(trainDataset, 
                     max_size = MAX_VOCAB_SIZE, 
                     vectors = "glove.6B.100d", 
                     unk_init = torch.Tensor.normal_)

Label.build_vocab(trainDataset)

.vector_cache/glove.6B.zip: 862MB [06:28, 2.22MB/s]                           
 99%|█████████▉| 397396/400000 [00:15<00:00, 26436.15it/s]

By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [35]:
print('Size of input vocab : ', len(Sentence.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Sentence.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  17213
Size of label vocab :  25
Top 10 words appreared repeatedly : [('.', 20129), (',', 17833), ('the', 15237), ('and', 11194), ('of', 11108), ('a', 11067), ('to', 7547), ('-', 6865), ("'s", 6354), ('is', 6326)]
Labels :  defaultdict(<function _default_unk_index at 0x7fe561fdd510>, {19: 0, 20: 1, 7: 2, 21: 3, 6: 4, 8: 5, 18: 6, 13: 7, 5: 8, 16: 9, 10: 10, 12: 11, 22: 12, 9: 13, 17: 14, 11: 15, 14: 16, 15: 17, 23: 18, 4: 19, 3: 20, 2: 21, 24: 22, 1: 23, 25: 24})


**Lots of stopwords!!**

Now we need to create a data loader to feed into our training loop. Torchtext provides the BucketIterator method that will produce what it calls a Batch, which is almost, but not quite, like the data loader we used on images.

But at first declare the device we are using.

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [37]:
train_iterator, valid_iterator = data.BucketIterator.splits((trainDataset, validDataset), batch_size = 256, 
                                                            sort_key = lambda x: len(x.sentence),
                                                            sort_within_batch=True, device = device)

Save the vocabulary for later use

In [38]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Sentence.vocab.stoi, tokens)

## Defining Our Model

We use the Embedding and LSTM modules in PyTorch to build a simple model for classifying tweets.

In this model we create three layers. 
1. First, the words in our tweets are pushed into an Embedding layer, which we have established as a 300-dimensional vector embedding. 
2. That’s then fed into a 2 stacked-LSTMs with 100 hidden features (again, we’re compressing down from the 300-dimensional input like we did with images). We are using 2 LSTMs for using the dropout.
3. Finally, the output of the LSTM (the final hidden state after processing the incoming tweet) is pushed through a standard fully connected layer with three outputs to correspond to our three possible classes (negative, positive, or neutral).

In [65]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout, bidirectional, pad_idx):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           bidirectional = bidirectional,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        self.dropout = nn.Dropout(dropout)
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.dropout(self.fc(hidden))
        #dense_outputs = self.fc(hidden)
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [66]:
# Define hyperparameters
size_of_vocab = len(Sentence.vocab)
embedding_dim = 100
num_hidden_nodes = 256
num_output_nodes = 25
num_layers = 3

dropout = 0.5
bidirectional = True
PAD_IDX = Sentence.vocab.stoi[Sentence.pad_token]

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout, bidirectional, PAD_IDX)

In [67]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(17213, 100, padding_idx=1)
  (encoder): LSTM(100, 256, num_layers=3, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=256, out_features=25, bias=True)
)
The model has 5,614,829 trainable parameters


## Model Training and Evaluation

First define the optimizer and loss functions

In [68]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc

In [69]:
pretrained_embeddings = Sentence.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([17213, 100])


In [70]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.0166, -0.4668,  2.0909,  ..., -1.4692,  0.4476, -0.7223],
        [-0.0791, -0.2089, -0.3442,  ...,  0.4657,  0.6297, -1.7395],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [-0.2031,  0.5047, -0.2522,  ..., -0.3462, -0.1863, -0.3161],
        [-0.7630,  0.7469, -0.1344,  ..., -0.4507, -0.0519, -0.1589],
        [-0.6918,  0.4727,  0.4129,  ..., -0.2911,  0.0815, -1.4222]])

In [71]:
UNK_IDX = Sentence.vocab.stoi[Sentence.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(embedding_dim)
model.embedding.weight.data[PAD_IDX] = torch.zeros(embedding_dim)
print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [-0.2031,  0.5047, -0.2522,  ..., -0.3462, -0.1863, -0.3161],
        [-0.7630,  0.7469, -0.1344,  ..., -0.4507, -0.0519, -0.1589],
        [-0.6918,  0.4727,  0.4129,  ..., -0.2911,  0.0815, -1.4222]])


In [72]:
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

The main thing to be aware of in this new training loop is that we have to reference `batch.tweets` and `batch.labels` to get the particular fields we’re interested in; they don’t fall out quite as nicely from the enumerator as they do in torchvision.

**Training Loop**

In [73]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        text, text_lengths = batch.sentence   
        
        # convert to 1D tensor
        predictions = model(text, text_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Evaluation Loop**

In [74]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            text, text_lengths = batch.sentence
            
            # convert to 1d tensor
            predictions = model(text, text_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Let's Train and Evaluate**

In [75]:
N_EPOCHS = 50
best_valid_loss = float('inf')

#freeze embeddings
model.embedding.weight.requires_grad = unfrozen = False

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 3.209 | Train Acc: 6.45%
	 Val. Loss: 3.202 |  Val. Acc: 5.86% 

	Train Loss: 3.204 | Train Acc: 6.75%
	 Val. Loss: 3.201 |  Val. Acc: 6.01% 

	Train Loss: 3.201 | Train Acc: 7.47%
	 Val. Loss: 3.198 |  Val. Acc: 7.11% 

	Train Loss: 3.196 | Train Acc: 8.13%
	 Val. Loss: 3.194 |  Val. Acc: 7.42% 

	Train Loss: 3.195 | Train Acc: 8.19%
	 Val. Loss: 3.191 |  Val. Acc: 8.20% 

	Train Loss: 3.194 | Train Acc: 8.23%
	 Val. Loss: 3.187 |  Val. Acc: 8.48% 

	Train Loss: 3.192 | Train Acc: 8.38%
	 Val. Loss: 3.187 |  Val. Acc: 8.77% 

	Train Loss: 3.191 | Train Acc: 8.78%
	 Val. Loss: 3.192 |  Val. Acc: 8.25% 

	Train Loss: 3.187 | Train Acc: 9.07%
	 Val. Loss: 3.183 |  Val. Acc: 10.83% 

	Train Loss: 3.186 | Train Acc: 9.20%
	 Val. Loss: 3.189 |  Val. Acc: 8.36% 

	Train Loss: 3.185 | Train Acc: 9.33%
	 Val. Loss: 3.183 |  Val. Acc: 10.83% 

	Train Loss: 3.186 | Train Acc: 9.07%
	 Val. Loss: 3.183 |  Val. Acc: 8.70% 

	Train Loss: 3.186 | Train Acc: 9.27%
	 Val. Loss: 3.185 |  Va

## Model Testing

In [50]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {1: "1", 2:"2", 3:"3", 4: "4",
                  5: "5", 6: "6" , 7:"7", 8:"8",
                  9:"9", 10:"10", 11:"11", 12:"12",
                  13:"13", 14:"14", 15:"15", 16:"16",
                  17:"17", 18: "18", 19:"19", 20:"20",
                  21:"21", 22:"22", 23:"23", 24:"24", 25:"25"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [ ]:
classify_tweet("A valid explanation for why Trump won't let women on the golf course.")

## Discussion on Data Augmentation Techniques 

You might wonder exactly how you can augment text data. After all, you can’t really flip it horizontally as you can an image! :D 

In contrast to data augmentation in images, augmentation techniques on data is very specific to final product you are building. As its general usage on any type of textual data doesn't provides a significant performance boost, that's why unlike torchvision, torchtext doesn’t offer a augmentation pipeline. Due to powerful models as transformers, augmentation tecnhiques are not so preferred now-a-days. But its better to know about some techniques with text that will provide your model with a little more information for training. 

### Synonym Replacement

First, you could replace words in the sentence with synonyms, like so:

    The dog slept on the mat

could become

    The dog slept on the rug

Aside from the dog's insistence that a rug is much softer than a mat, the meaning of the sentence hasn’t changed. But mat and rug will be mapped to different indices in the vocabulary, so the model will learn that the two sentences map to the same label, and hopefully that there’s a connection between those two words, as everything else in the sentences is the same.

### Random Insertion
A random insertion technique looks at a sentence and then randomly inserts synonyms of existing non-stopwords into the sentence n times. Assuming you have a way of getting a synonym of a word and a way of eliminating stopwords (common words such as and, it, the, etc.), shown, but not implemented, in this function via get_synonyms() and get_stopwords(), an implementation of this would be as follows:


In [51]:
def random_insertion(sentence, n): 
    words = remove_stopwords(sentence) 
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym) 
    return sentence

## Random Deletion
As the name suggests, random deletion deletes words from a sentence. Given a probability parameter p, it will go through the sentence and decide whether to delete a word or not based on that random probability. Consider of it as pixel dropouts while treating images.

In [52]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

### Random Swap
The random swap augmentation takes a sentence and then swaps words within it n times, with each iteration working on the previously swapped sentence. Here we sample two random numbers based on the length of the sentence, and then just keep swapping until we hit n.

In [53]:
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

For more on this please go through this [paper](https://arxiv.org/pdf/1901.11196.pdf).

### Back Translation

Another popular approach for augmenting text datasets is back translation. This involves translating a sentence from our target language into one or more other languages and then translating all of them back to the original language. We can use the Python library googletrans for this purpose. 

In [ ]:
!!pip install googletrans==3.1.0a0


In [ ]:
import random
import googletrans
from googletrans import Translator
#import googletrans.Translator

translator = Translator()
sentence = ['The dog slept on the rug']

available_langs = list(googletrans.LANGUAGES.keys()) 
trans_lang = random.choice(available_langs) 
print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")

translations = translator.translate(sentence, dest=trans_lang) 
t_text = [t.text for t in translations]
print(t_text)

translations_en_random = translator.translate(t_text, src=trans_lang, dest='en') 
en_text = [t.text for t in translations_en_random]
print(en_text)